In [1]:
import ast
import glob
import os
import random
import re

import numpy as np
import pandas as pd

from __future__ import annotations
from codealltag_data_processor_v2025 import CodealltagDataProcessor
from concurrent.futures import ThreadPoolExecutor, as_completed
from pandas import DataFrame
from pandas.core.series import Series
from tqdm import tqdm
from typing import Any, Dict, Generator, List, Tuple

In [2]:
cdp_2022 = CodealltagDataProcessor(data_version='20220513', config_path=['codealltag_data_processor.yml'])

In [3]:
predicted_text_df = pd.concat(
    [
        pd.read_csv(df, index_col=0) 
        for df in glob.glob(os.path.join('.', f'PredictedText_DF_{cdp_2022.get_data_version()}_15K_*.csv'))
    ],
    axis=0,
    ignore_index=True
)

In [4]:
labels = ['CITY', 'DATE', 'EMAIL', 'FAMILY', 'FEMALE', 'MALE', 'ORG', 
          'PHONE', 'STREET', 'STREETNO', 'UFID', 'URL', 'USER', 'ZIP']

In [5]:
def get_annotation_df_with_input_text_and_predicted_text(input_text: str, 
                                                         predicted_text: str,
                                                         labels: List[str]) -> DataFrame:
    tuples = list()

    input_text_length = len(input_text)
    input_text_copy = input_text[0: input_text_length]

    item_delim = "; "
    token_delim = ": "
    pseudonym_delim = " **"
    token_id = 0
    next_cursor = 0

    predicted_items = predicted_text.split(item_delim)
    for item in predicted_items:

        label, token, pseudonym = "", "", ""

        for l in labels:
            if item.startswith(l):
                label = l

        if label != "" and (label+token_delim) in item:

            value_splits = item.split(label+token_delim)
            token_pseudonym = value_splits[1]

            if (pseudonym_delim in token_pseudonym and token_pseudonym.endswith(pseudonym_delim.strip())):

                pseudonym_splits = token_pseudonym.split(pseudonym_delim)
                token = pseudonym_splits[0]
                pseudonym = pseudonym_splits[1][:-2]

            else:
                token = token_pseudonym

            if len(token.strip()) > 0:

                start = input_text_copy.find(token)
                if start == -1 and ' ' in token:
                    start = input_text_copy.find(token.split(' ')[0])
                    token = token.replace(' ', '')

                if start != -1:
                    end = start + len(token)

                    token_id += 1
                    prev_cursor = next_cursor
                    next_cursor += end
                    input_text_copy = input_text[next_cursor: input_text_length]

                    start = prev_cursor + start
                    end = prev_cursor + end

                    tuples.append((
                        'T' + str(token_id),
                        label,
                        start,
                        end,
                        input_text[start:end],
                        pseudonym
                    ))

    return pd.DataFrame(
        tuples,
        columns=["Token_ID", "Label", "Start", "End", "Token", "Pseudonym"]
    )

In [6]:
def get_pseudonymized_text(input_text: str, predicted_annotation_df: DataFrame) -> str:
    output_text = input_text
    offset = 0
    for index, row in predicted_annotation_df.iterrows():
        output_text = output_text[:(row.Start+offset)] + row.Pseudonym + output_text[(row.End+offset):]
        offset += len(row.Pseudonym) - len(row.Token)
    return output_text

In [7]:
def get_annotation_df_for_pseudonymized_text(pseudonymized_text: str, pseudonymized_annotation_df: DataFrame) -> DataFrame:
    tuples = list()
    last_cursor = 0
    for index, row in pseudonymized_annotation_df.iterrows():
        pseudonym = row.Pseudonym
        start = pseudonymized_text[last_cursor:].find(pseudonym)
        if start != -1:
            tuples.append((row.Token_ID, row.Label, last_cursor+start, last_cursor+start+len(pseudonym), pseudonym))
            last_cursor += (start + len(pseudonym))
    return pd.DataFrame(
        tuples,
        columns=["Token_ID", "Label", "Start", "End", "Token"]
    )

In [8]:
def check_for_exact_match_and_prepare_entry_for_new_dataset(predicted_text_df: DataFrame, 
                                                            idx: int, 
                                                            cdp: CodealltagDataProcessor) -> Dict[str, Tuple]:
    
    row: Series = predicted_text_df.iloc[idx]
    file_path: str = row.FilePath
    original_text: str = cdp.read_email(file_path)[1]
    original_adf: DataFrame = cdp.get_annotation_df_by_file(file_path)
    original_lt_str: str = '|'.join(original_adf.Label + '-' + original_adf.Token)
    versions: List[str] = [col for col in predicted_text_df.columns if re.match(r'^V\d+$', col)]
    version_match_list: List[Tuple] = list()
    for version in versions:
        version_adf: DataFrame = get_annotation_df_with_input_text_and_predicted_text(original_text, row[version], labels)
        version_lt_str: str = '|'.join(version_adf.Label + '-' + version_adf.Token)
        if original_lt_str == version_lt_str and (version_adf.Token != version_adf.Pseudonym).all():
            version_match_list.append((version, row[version], version_adf))
    if len(version_match_list) == 0:
        return {file_path: None}
    version_tuple = None
    if len(version_match_list) == 1:
        version_tuple = version_match_list[0]
    else:
        version_tuple = random.Random(cdp.get_random_seed()).choice(version_match_list)
    original_tokenized_text = " ".join(cdp.tokenize_with_somajo(original_text))
    original_tokenized_text_with_label = cdp.tokenize_with_somajo_and_annotation_df(original_text, original_adf)
    mT5_output = version_tuple[1]
    mT5_adf = version_tuple[2]
    mT5_pseudonymized_text = get_pseudonymized_text(original_text, mT5_adf)
    mT5_pseudonymized_tokenized_text = " ".join(cdp.tokenize_with_somajo(mT5_pseudonymized_text))
    mT5_s_adf = get_annotation_df_for_pseudonymized_text(mT5_pseudonymized_text, mT5_adf)
    mT5_pseudonymized_tokenized_text_with_label = cdp.tokenize_with_somajo_and_annotation_df(mT5_pseudonymized_text, mT5_s_adf)
    output_tuple = (
        file_path,
        original_text,
        original_adf.drop(columns="FilePath").to_dict(),
        original_tokenized_text,
        original_tokenized_text_with_label,
        mT5_output,
        mT5_pseudonymized_text,
        mT5_adf.to_dict(),
        mT5_pseudonymized_tokenized_text,
        mT5_pseudonymized_tokenized_text_with_label
    )
    return {file_path: output_tuple}

In [9]:
def collect_entries_for_new_dataset(max_workers: int = 10) -> Generator[Dict[str, Tuple]]:
    with tqdm(total=len(predicted_text_df), smoothing=0) as progress_bar:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = [
                executor.submit(check_for_exact_match_and_prepare_entry_for_new_dataset, predicted_text_df, idx, cdp_2022)
                for idx in range(0, len(predicted_text_df))
            ]
            for future in as_completed(futures):
                progress_bar.update(1)
                yield future.result()

In [10]:
merged_dict: Dict[str, Tuple] = {}
for result in collect_entries_for_new_dataset():
    merged_dict.update(result)

100%|███████████████████████████████████| 15000/15000 [2:47:31<00:00,  1.49it/s]


In [11]:
tuples = [merged_dict[fp] for fp in merged_dict.keys() if merged_dict[fp]]
columns = ["FilePath", "OT", "OA", "OTT", "OTTL", "MT5O", "MT5PT", "MT5PA", "MT5PTT", "MT5PTTL"]
dataset_df = pd.DataFrame(tuples, columns)
dataset_df.to_csv("data_utility_dataset.csv")

In [12]:
dataset = pd.read_csv("data_utility_dataset.csv", index_col=0)

In [70]:
idx = 177

In [71]:
print(dataset.iloc[idx].OT)

***Siegfried T.  (whoqob) schrieb:

[...]

im Grunde genommen interessiert es mich nicht, ich schau auf den Chart  
:-)

Muß allerdings eingestehen, das ich, aufgeheizt durch diese Group einig  
zig davon, zum Glück verkehrt geodert und somit nicht bekommen habe.

Keine Angst.... dafür habe ich in eine andere Kloschüssel gegriffen...


  Gruss  Benjamin

PS. Nun warte ich auf den Klempner.... und dann wird es schon wieder im  
Abfluß brodeln ;-)


In [72]:
pd.DataFrame(ast.literal_eval(dataset.iloc[idx].OA))

,Token_ID,Label,Start,End,Token
0,T1,MALE,3,12,Siegfried
1,T2,FAMILY,13,15,T.
2,T3,USER,18,24,whoqob
3,T4,MALE,347,355,Benjamin


In [73]:
print(dataset.iloc[idx].OTT)

* * * Siegfried T. ( whoqob ) schrieb : [ ... ] im Grunde genommen interessiert es mich nicht , ich schau auf den Chart :-) Muß allerdings eingestehen , das ich , aufgeheizt durch diese Group einig zig davon , zum Glück verkehrt geodert und somit nicht bekommen habe . Keine Angst .... dafür habe ich in eine andere Kloschüssel gegriffen ... Gruss Benjamin PS. Nun warte ich auf den Klempner .... und dann wird es schon wieder im Abfluß brodeln ;-)


In [74]:
print(dataset.iloc[idx].OTTL)

* O
* O
* O
Siegfried B-MALE
T. B-FAMILY
( O
whoqob B-USER
) O
schrieb O
: O
[ O
... O
] O
im O
Grunde O
genommen O
interessiert O
es O
mich O
nicht O
, O
ich O
schau O
auf O
den O
Chart O
:-) O
Muß O
allerdings O
eingestehen O
, O
das O
ich O
, O
aufgeheizt O
durch O
diese O
Group O
einig O
zig O
davon O
, O
zum O
Glück O
verkehrt O
geodert O
und O
somit O
nicht O
bekommen O
habe O
. O
Keine O
Angst O
.... O
dafür O
habe O
ich O
in O
eine O
andere O
Kloschüssel O
gegriffen O
... O
Gruss O
Benjamin B-MALE
PS. O
Nun O
warte O
ich O
auf O
den O
Klempner O
.... O
und O
dann O
wird O
es O
schon O
wieder O
im O
Abfluß O
brodeln O
;-) O



In [75]:
print(dataset.iloc[idx].MT5O)

MALE: Siegfried **Ottmar**; FAMILY: T. **W.**; USER: whoqob **knkqjm**; MALE: Benjamin **Vinzenz**


In [76]:
print(dataset.iloc[idx].MT5PT)

***Ottmar W.  (knkqjm) schrieb:

[...]

im Grunde genommen interessiert es mich nicht, ich schau auf den Chart  
:-)

Muß allerdings eingestehen, das ich, aufgeheizt durch diese Group einig  
zig davon, zum Glück verkehrt geodert und somit nicht bekommen habe.

Keine Angst.... dafür habe ich in eine andere Kloschüssel gegriffen...


  Gruss  Vinzenz

PS. Nun warte ich auf den Klempner.... und dann wird es schon wieder im  
Abfluß brodeln ;-)


In [77]:
pd.DataFrame(ast.literal_eval(dataset.iloc[idx].MT5PA))

,Token_ID,Label,Start,End,Token,Pseudonym
0,T1,MALE,3,12,Siegfried,Ottmar
1,T2,FAMILY,13,15,T.,W.
2,T3,USER,18,24,whoqob,knkqjm
3,T4,MALE,347,355,Benjamin,Vinzenz


In [78]:
print(dataset.iloc[idx].MT5PTT)

* * * Ottmar W. ( knkqjm ) schrieb : [ ... ] im Grunde genommen interessiert es mich nicht , ich schau auf den Chart :-) Muß allerdings eingestehen , das ich , aufgeheizt durch diese Group einig zig davon , zum Glück verkehrt geodert und somit nicht bekommen habe . Keine Angst .... dafür habe ich in eine andere Kloschüssel gegriffen ... Gruss Vinzenz PS. Nun warte ich auf den Klempner .... und dann wird es schon wieder im Abfluß brodeln ;-)


In [79]:
print(dataset.iloc[idx].MT5PTTL)

* O
* O
* O
Ottmar B-MALE
W. B-FAMILY
( O
knkqjm B-USER
) O
schrieb O
: O
[ O
... O
] O
im O
Grunde O
genommen O
interessiert O
es O
mich O
nicht O
, O
ich O
schau O
auf O
den O
Chart O
:-) O
Muß O
allerdings O
eingestehen O
, O
das O
ich O
, O
aufgeheizt O
durch O
diese O
Group O
einig O
zig O
davon O
, O
zum O
Glück O
verkehrt O
geodert O
und O
somit O
nicht O
bekommen O
habe O
. O
Keine O
Angst O
.... O
dafür O
habe O
ich O
in O
eine O
andere O
Kloschüssel O
gegriffen O
... O
Gruss O
Vinzenz B-MALE
PS. O
Nun O
warte O
ich O
auf O
den O
Klempner O
.... O
und O
dann O
wird O
es O
schon O
wieder O
im O
Abfluß O
brodeln O
;-) O

